### Data cleaning:
1) Salary parsing: Removing '(Glassdoor est.)', 'K' and '$' / Adding new columns to indicate if the salary is given per hour or not, the min and max salaries and the averages

2) Adding a State field

3) Adding the Age of the company

4) Job Description parsing

5) Job Title parsing

6) Making hourly salary to annual

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("glassdoor_job_offers1.csv")
df2 = pd.read_csv("glassdoor_job_offers2.csv")
df3 = pd.read_csv("glassdoor_job_offers3.csv")
df4 = pd.read_csv("glassdoor_job_offers4.csv")

In [3]:
data = pd.concat([df1, df2, df3, df4])

In [4]:
data.shape

(330, 12)

In [5]:
# Remove duplicated observations
print(data.duplicated().sum())
data.drop_duplicates(inplace=True)

# Remove rows with salary value equal to -1
data = data[data["Salary Estimate"] != '-1'].reset_index(drop=True)

90


In [6]:
# Salary parsing

# Remove '(Glassdoor est.)'
salary = data["Salary Estimate"].apply(lambda x: x.split('(')[0])

# Remove 'K' and '$'
salary = salary.apply(lambda x: x.replace('K', '').replace('$', ''))

# Some salaries are given per hour, so let's make a new column to indicate if the salary is per hour or not
data['Per_hour'] = data['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
salary = salary.apply(lambda x: x.replace('Per Hour', ''))

data['Salary'] = salary
data['Min_salary'] = salary.apply(lambda x: int(x.split('-')[0]))
data['Max_salary'] = salary.apply(lambda x: int(x.split('-')[1]))
data['Average_salary'] = (data['Min_salary'] + data['Max_salary'])/2

In [7]:
# State field
print( data['Location'])
data['Job_State'] = data['Location'].apply(lambda x: x.split(',')[1] if ',' in x else x)


0       Alexandria, VA
1          Chicago, IL
2        Middleton, WI
3           Lorton, VA
4            Ogden, UT
            ...       
220             Remote
221         Draper, UT
222             Remote
223    Saint Louis, MO
224             Remote
Name: Location, Length: 225, dtype: object


In [8]:
# Add the age of the company
data['Company_age'] = data['Founded'].apply(lambda x: (2021-x) if x>0 else x)


In [9]:
# Job Description parsing
data['Python_yn'] = data['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
print(data['Python_yn'].value_counts())

data['AI_yn'] = data['Job Description'].apply(lambda x: 1 if 'ai' in x.lower() else 0)
print(data['AI_yn'].value_counts())

data['ML_yn'] = data['Job Description'].apply(lambda x: 1 if 'ml' in x.lower() else 0)
print(data['ML_yn'].value_counts())

data['Statistic_yn'] = data['Job Description'].apply(lambda x: 1 if 'statistic' in x.lower() else 0)
print(data['Statistic_yn'].value_counts())

0    217
1      8
Name: Python_yn, dtype: int64
1    122
0    103
Name: AI_yn, dtype: int64
0    217
1      8
Name: ML_yn, dtype: int64
0    182
1     43
Name: Statistic_yn, dtype: int64


In [14]:
# Add Job Description length
data['desc_len'] = data['Job Description'].apply(lambda x: len(x))

In [10]:
data

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,Salary,Min_salary,Max_salary,Average_salary,Job_State,Company_age,Python_yn,AI_yn,ML_yn,Statistic_yn
0,Data Scientist,$64K - $106K (Glassdoor est.),"Secure our Nation, Ignite your Future\nOvervie...",4.2,ManTech International Corporation,"Alexandria, VA",5001 to 10000 Employees,1968,Company - Public,Research & Development,...,64 - 106,64,106,85.0,VA,53,0,1,0,0
1,Sr. Data Scientist,$83K - $138K (Glassdoor est.),Sr. Data Scientist -INF0001YE\nDescription\n\n...,3.6,CNA Insurance,"Chicago, IL",1001 to 5000 Employees,1897,Company - Public,Insurance Carriers,...,83 - 138,83,138,110.5,IL,124,0,0,0,0
2,R&D Scientist,$38K - $68K (Glassdoor est.),"LGC, Biosearch Technologies is seeking a detai...",3.5,LGC Limited,"Middleton, WI",1001 to 5000 Employees,1842,Company - Private,Biotech & Pharmaceuticals,...,38 - 68,38,68,53.0,WI,179,0,1,0,0
3,Data Scientist,$71K - $119K (Glassdoor est.),Who Are We?\nQinetiQ Inc. is a leading innovat...,2.5,QinetiQ Inc.,"Lorton, VA",501 to 1000 Employees,1956,Subsidiary or Business Segment,Aerospace & Defense,...,71 - 119,71,119,95.0,VA,65,0,1,1,0
4,Data Engineer,$44K - $87K (Glassdoor est.),Schedule: Mon- Fri Overview:\nThe data enginee...,3.8,America First Credit Union,"Ogden, UT",1001 to 5000 Employees,1939,Nonprofit Organization,Investment Banking & Asset Management,...,44 - 87,44,87,65.5,UT,82,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,NLP - MRP Data Scientist,$100K - $200K (Employer est.),NLP-MRP Data Scientist\nLocation: Remote\nPurp...,4.3,Espire Services,Remote,1 to 50 Employees,-1,Contract,-1,...,100 - 200,100,200,150.0,Remote,-1,0,0,0,0
221,Data Scientist / Malware Researcher,$100K - $200K (Employer est.),"Please Note:\n1. If you are a first time user,...",3.2,Broadcom,"Draper, UT",10000+ Employees,1991,Company - Public,Electrical & Electronic Manufacturing,...,100 - 200,100,200,150.0,UT,30,0,1,0,0
222,Data Scientist - Senior Health Solutions - Remote,$100K - $200K (Employer est.),This position is on the data science team with...,4.3,Mutual of Omaha,Remote,5001 to 10000 Employees,1909,Company - Private,Insurance Carriers,...,100 - 200,100,200,150.0,Remote,112,0,0,0,0
223,Data Scientist - Intermediate,$100K - $200K (Employer est.),KellyMitchell matches the best IT and business...,4.1,KellyMitchell,"Saint Louis, MO",1001 to 5000 Employees,1998,Company - Private,Staffing & Outsourcing,...,100 - 200,100,200,150.0,MO,23,1,0,0,1


In [11]:
# Job title and seniority
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'machine learning' in title.lower():
        return 'ml'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'not_specified'
    
    
def seniority(title):
    if 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'junior' in title.lower() or 'jr' in title.lower():
        return 'junior'
    else:
        return 'not_specified'

In [12]:
data['Simplified_title'] = data['Job Title'].apply(title_simplifier)

In [13]:
data['Seniority'] = data['Job Title'].apply(seniority)

In [18]:
# Make hourly salary to annual
# (2 = 2000/1000) and 2000 = 8h/day * 5d/week * 50w/year
data['Min_salary'] = data.apply(lambda x: x.Min_salary*2 if x.Per_hour==1 else x.Min_salary, axis=1)
data['Max_salary'] = data.apply(lambda x: x.Max_salary*2 if x.Per_hour==1 else x.Max_salary, axis=1)

In [28]:
data.to_csv("cleaned_data.csv", index=False)